In [ ]:
from evidently import Report
from evidently.presets import DataDriftPreset, DataSummaryPreset

from evidently.ui.workspace import Workspace
from evidently.sdk.panels import *
from evidently.sdk.models import PanelMetric
from evidently.legacy.renderers.html_widgets import WidgetSize
from evidently import DataDefinition
from evidently import Dataset
from evidently import Report
from evidently.metrics import (
    ValueDrift, 
    DriftedColumnsCount,
    MissingValueCount,
    StdValue,
    QuantileValue
    )
import pandas as pd
import datetime as dt

import joblib

In [4]:
mar_data = pd.read_parquet('data/green_tripdata_2024-03.parquet')

In [5]:
mar_data.describe()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
count,57457.000000,57457,57457,55360.000000,57457.000000,57457.000000,55360.000000,57457.000000,57457.000000,57457.000000,57457.000000,57457.000000,57457.000000,0.0,57457.000000,57457.000000,55360.000000,55353.000000,55360.000000
mean,1.877334,2024-03-16 04:02:52.405399,2024-03-16 04:21:00.076039,1.179986,95.524688,138.629149,1.309538,13.522828,17.313474,0.904472,0.577410,2.386255,0.192537,NaN,0.979378,22.904832,1.321062,1.038047,0.737730
min,1.000000,2008-12-31 23:02:24,2008-12-31 23:02:30,1.000000,1.000000,1.000000,0.000000,0.000000,-295.080000,-2.500000,-0.500000,-1.560000,0.000000,NaN,-1.000000,-296.080000,1.000000,1.000000,-2.750000
25%,2.000000,2024-03-08 13:53:56,2024-03-08 14:13:49,1.000000,74.000000,74.000000,1.000000,1.100000,9.300000,0.000000,0.500000,0.000000,0.000000,NaN,1.000000,13.440000,1.000000,1.000000,0.000000
50%,2.000000,2024-03-15 22:49:01,2024-03-15 23:09:52,1.000000,75.000000,138.000000,1.000000,1.790000,13.500000,0.000000,0.500000,2.000000,0.000000,NaN,1.000000,18.500000,1.000000,1.000000,0.000000
75%,2.000000,2024-03-23 20:11:25,2024-03-23 20:34:48,1.000000,97.000000,220.000000,1.000000,3.100000,19.800000,1.000000,0.500000,3.610000,0.000000,NaN,1.000000,27.050000,2.000000,1.000000,2.750000
max,2.000000,2024-04-01 00:01:45,2024-04-01 16:11:00,99.000000,265.000000,265.000000,9.000000,125112.200000,841.600000,10.000000,4.250000,150.000000,26.760000,NaN,1.000000,856.980000,5.000000,2.000000,2.750000
std,0.328056,NaN,NaN,1.356719,57.285088,76.295346,0.967749,770.416255,14.958249,1.382446,0.366916,3.159273,1.184551,NaN,0.154253,17.013735,0.497858,0.191311,1.218039


In [6]:
mar_data.shape

(57457, 20)

In [7]:
mar_data["duration_min"] = mar_data.lpep_dropoff_datetime - mar_data.lpep_pickup_datetime
mar_data.duration_min = mar_data.duration_min.apply(lambda td : float(td.total_seconds())/60)

In [8]:
# filter out outliers
mar_data = mar_data[(mar_data.duration_min >= 0) & (mar_data.duration_min <= 60)]
mar_data = mar_data[(mar_data.passenger_count > 0) & (mar_data.passenger_count <= 8)]

In [9]:
target = "duration_min"
num_features = ["passenger_count", "trip_distance", "fare_amount", "total_amount"]
cat_features = ["PULocationID", "DOLocationID"]

### Creating prediction column

In [10]:
model = joblib.load("models/lin_reg.bin")
mar_data["prediction"] = model.predict(mar_data[num_features + cat_features])

### Evidently Dashboard

In [11]:
ws = Workspace(path="workspace")
project = ws.create_project("HW 5 - NYC Taxi Data")
project.description = "MLOps Zoomcamp - HW5 - Evidently"
project.save()

In [26]:
data_definition = DataDefinition(numerical_columns = num_features + ['prediction'],
                                 categorical_columns=cat_features)

report = Report(metrics=[
    StdValue(column='prediction'),
    QuantileValue(column='fare_amount', quantile=0.5),
]
)

for i in range(1,29):
    print(f"Processing data for 2024-03-{i:02d}")
    
    if len(mar_data.loc[mar_data.lpep_pickup_datetime.between(
        f'2024-03-{i:02d}', 
        f'2024-03-{i+1:02d}', 
        inclusive="left")]) == 0:
        print(f"No data for 2024-03-{i:02d}")
        continue
    
    day_data = mar_data.loc[mar_data.lpep_pickup_datetime.between(
        f'2024-03-{i:02d}', 
        f'2024-03-{i+1:02d}', 
        inclusive="left")]

    data = Dataset.from_pandas(data=day_data,
                            data_definition = data_definition)

    snapshot = report.run(reference_data=mar_data,
                current_data=data,
                timestamp=dt.datetime(2024, 3, i))
    
    ws.add_run(project.id, snapshot)

Processing data for 2024-03-01
Processing data for 2024-03-02


/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide

/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide



Processing data for 2024-03-03
Processing data for 2024-03-04
Processing data for 2024-03-05


/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide

/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide

/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide



Processing data for 2024-03-06
Processing data for 2024-03-07
Processing data for 2024-03-08


...

/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide

/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide



Processing data for 2024-03-23
Processing data for 2024-03-24
Processing data for 2024-03-25


/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide

/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide

/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide



Processing data for 2024-03-26
Processing data for 2024-03-27
Processing data for 2024-03-28


/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide

/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide

/home/gus/miniconda3/envs/monitoring/lib/python3.11/site-packages/evidently/core/datasets.py:990: RuntimeWarning:

invalid value encountered in scalar divide



In [28]:
project.dashboard.clear_dashboard()

project.dashboard.add_panel(
    DashboardPanelPlot(
        title="Homework 5 - NYC Taxi Data",
        size="full",
        values=[],
        plot_params={"plot_type": "text"}
    )
)

project.dashboard.add_panel(
    DashboardPanelPlot(
        title="Prediction Standard Deviation",
        size="half",
        values=[
            PanelMetric(legend="std", metric="StdValue", metric_labels={"column":"prediction"})
        ]
    )
)

project.dashboard.add_panel(
    DashboardPanelPlot(
        title="Median Fare Amount",
        size="half",
        values=[
            PanelMetric(legend="median", metric="QuantileValue", metric_labels={"column":"fare_amount", "quantile":0.5})
        ]
    )
)

